# Wave2Vec2
https://huggingface.co/docs/transformers/model_doc/wav2vec2

In [1]:
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC
from datasets import load_dataset, Audio
import datasets
import torch

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject


In [2]:
import os
hfhome_dir=os.path.join(os.sep, 'DATA10T', 'Cache')
#os.environ['TRANSFORMERS_CACHE'] = hfhome_dir
os.environ['HF_HOME'] = hfhome_dir
#os.environ['HF_HUB_CACHE'] = os.path.join(hfhome_dir, 'hub')
os.environ['HF_DATASETS_CACHE'] = hfhome_dir
#HF_HUB_OFFLINE=1
mycache_dir = hfhome_dir

https://huggingface.co/facebook/mms-1b-all

In [4]:
dataset_name = "mozilla-foundation/common_voice_11_0"

# English
stream_data = load_dataset(dataset_name, "en", split="test", cache_dir=mycache_dir, streaming=True) #
stream_data = stream_data.cast_column("audio", Audio(sampling_rate=16000))
en_sample = next(iter(stream_data))["audio"]["array"]

# French
stream_data = load_dataset(dataset_name, "fr", split="test", cache_dir=mycache_dir, streaming=True) #streaming=True
stream_data = stream_data.cast_column("audio", Audio(sampling_rate=16000))
fr_sample = next(iter(stream_data))["audio"]["array"]

# Chinese
stream_data = load_dataset(dataset_name, "zh-CN", split="test", cache_dir=mycache_dir, streaming=True) #streaming=True
stream_data = stream_data.cast_column("audio", Audio(sampling_rate=16000))
zh_sample = next(iter(stream_data))["audio"]["array"]

/home/lkk/miniconda3/envs/mycondapy310/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Reading metadata...: 16354it [00:00, 18240.44it/s]
/home/lkk/miniconda3/envs/mycondapy310/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the 

In [35]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

import re

def gettestdata(dataset_name, language='en', split="test", sampling_rate=16000, streaming=False, samples = 10, task_column="audio", target_column="sentence"):
    if samples and samples>0:
        splitwithsample=split+f"[:{samples}]"
    else:
        splitwithsample=split
    dataset_test = load_dataset(dataset_name, language, split=splitwithsample, cache_dir=mycache_dir, streaming=streaming) #streaming=True

    #remove other columns
    column_names = dataset_test.column_names
    columnnames_remove = []
    for columnname in column_names:
        if columnname not in [task_column, target_column]:
            columnnames_remove.append(columnname)
    dataset_test = dataset_test.remove_columns(columnnames_remove)
    #dataset_test = dataset_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
    dataset_test = dataset_test.cast_column("audio", Audio(sampling_rate=sampling_rate))

    chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'
    def remove_special_characters(batch):
        batch[target_column] = re.sub(chars_to_ignore_regex, '', batch[target_column]).lower() + " "
        return batch
    dataset_test = dataset_test.map(remove_special_characters)
    
    return dataset_test
    #zh_sample = next(iter(dataset_test))["audio"]["array"]

In [40]:
dataset_name = "mozilla-foundation/common_voice_11_0"
dataset_en = gettestdata(dataset_name=dataset_name, language='en', split="test", sampling_rate=16000, streaming=False)

In [42]:
dataset_zh = gettestdata(dataset_name=dataset_name, language='zh-CN', split="test", sampling_rate=16000, streaming=False)

Generating train split: 0 examples [00:00, ? examples/s]

Reading metadata...: 29056it [00:00, 435552.25it/s]


Generating validation split: 0 examples [00:00, ? examples/s]

Reading metadata...: 10581it [00:00, 465530.26it/s]


Generating test split: 0 examples [00:00, ? examples/s]

...

Generating other split: 0 examples [00:00, ? examples/s]

Reading metadata...: 698486it [00:01, 493583.12it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]

Reading metadata...: 21302it [00:00, 484376.19it/s]


Map:   0%|          | 0/10581 [00:00<?, ? examples/s]

In [43]:
dataset_en

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 16354
})

In [44]:
dataset_zh

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 10581
})

In [45]:
dataset_en[0]

{'path': '/DATA10T/Cache/downloads/extracted/ba5c265b423696aeece652c80df565d72d9f7c1cc9b807e127f44f989150f409/en_test_0/common_voice_en_27710027.mp3',
 'audio': {'path': '/DATA10T/Cache/downloads/extracted/ba5c265b423696aeece652c80df565d72d9f7c1cc9b807e127f44f989150f409/en_test_0/common_voice_en_27710027.mp3',
  'array': array([-2.09547579e-09, -1.39698386e-09, -1.86264515e-09, ...,
          7.56020654e-07,  8.43494490e-07,  1.62231663e-06]),
  'sampling_rate': 16000},
 'sentence': 'joe keaton disapproved of films and buster also had reservations about the medium '}

In [46]:
import IPython.display as ipd
import numpy as np
import random
dataset_test = dataset_zh
target_column = "sentence"
task_column = "audio"
samplerate = 16000
rand_int = random.randint(0, len(dataset_test)-1)

print(dataset_test[rand_int][target_column])
ipd.Audio(data=dataset_test[rand_int][task_column]["array"], autoplay=True, rate=samplerate)

该文物保护单位由汪清县文物管理所管理。 


In [6]:
en_sample.shape

(100800,)

In [7]:
fr_sample.shape

(61056,)

In [8]:
zh_sample.shape

(38016,)

In [48]:
from transformers import Wav2Vec2ForCTC, AutoProcessor, AutoModelForCTC, AutoModelForAudioClassification
import torch

def loaddefaultmodel_fromname(modelname="facebook/wav2vec2-base-960h", task="audio-asr", cache_dir="./output"):
    if task == "audio-classification":
        feature_extractor = AutoFeatureExtractor.from_pretrained(modelname, cache_dir=cache_dir)
        model = AutoModelForAudioClassification.from_pretrained(modelname, cache_dir=cache_dir)
        tokenizer = None
        processor = None
    elif task == "audio-asr":
        model = AutoModelForCTC.from_pretrained(modelname, cache_dir=cache_dir)
        tokenizer = AutoTokenizer.from_pretrained(modelname, cache_dir=cache_dir)
        feature_extractor = AutoFeatureExtractor.from_pretrained(modelname, cache_dir=cache_dir)
        processor = AutoProcessor.from_pretrained(modelname, cache_dir=cache_dir)
    return model, feature_extractor, tokenizer, processor

In [49]:
from transformers import Wav2Vec2ForCTC, AutoProcessor
import torch

model_id = "facebook/mms-1b-all"

#processor = AutoProcessor.from_pretrained(model_id, cache_dir=mycache_dir)
#model = Wav2Vec2ForCTC.from_pretrained(model_id, cache_dir=mycache_dir)

model, feature_extractor, tokenizer, processor = loaddefaultmodel_fromname(modelname=model_id, task="audio-asr", cache_dir=mycache_dir)

Some weights of the model checkpoint at facebook/mms-1b-all were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/mms-1b-all and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream

In [58]:
import librosa
def asrinference_path(datasample, model, samplerate=16_000, processor=None, device='cuda'):
    if isinstance(datasample, str):#batch["path"] get numpyarray
        datasamples, sampling_rate = librosa.load(datasample, sr=samplerate)
        batchdecode=False
    elif isinstance(datasample, list):
        datasamples=[]
        for sample in datasample:
            onespeech, sampling_rate = librosa.load(sample, sr=samplerate)
            datasamples.append(onespeech)
        batchdecode=True
    
    inputs = processor(datasamples, sampling_rate=samplerate, return_tensors="pt", padding=True)
    print("Input:", type(inputs))

    model=model.to(device)
    inputs=inputs.to(device)

    with torch.no_grad():
        outputs = model(**inputs).logits

    print("Output logits shape:", outputs.shape)
    if batchdecode:
        predicted_ids = torch.argmax(outputs, dim=-1)
        transcription = processor.batch_decode(predicted_ids)
    else:
        ids = torch.argmax(outputs, dim=-1)[0]
        transcription = processor.decode(ids)
    return transcription

In [61]:
oneexample = dataset_en[rand_int]
print("Text:", oneexample['sentence'])
transcription = asrinference_path(oneexample["path"], model, samplerate=16_000, processor=processor, device='cuda')
transcription

Text: book spot in north catasauqua south dakota in sixteen minutes 
Input: <class 'transformers.feature_extraction_utils.BatchFeature'>
Output logits shape: torch.Size([1, 496, 154])


'books pot in north catasakua south dacota in sixteen minutes'

In [62]:
processor.tokenizer.set_target_lang("cmn-script_simplified")
model.load_adapter("cmn-script_simplified")

In [63]:
oneexample = dataset_zh[rand_int]
print("Text:", oneexample['sentence'])
transcription = asrinference_path(oneexample["path"], model, samplerate=16_000, processor=processor, device='cuda')
transcription

Text: 该文物保护单位由汪清县文物管理所管理。 
Input: <class 'transformers.feature_extraction_utils.BatchFeature'>
Output logits shape: torch.Size([1, 315, 4495])


'该文物保护单位由汪心县文物管理所管理'

In [64]:
test=dataset_zh[1,2,3]

In [65]:
test

{'path': ['/DATA10T/Cache/downloads/extracted/1e09a698341d47a1339fd58caa794de0162bda91e28f62daa9f56221d76d564b/zh-CN_test_0/common_voice_zh-CN_22069600.mp3',
  '/DATA10T/Cache/downloads/extracted/1e09a698341d47a1339fd58caa794de0162bda91e28f62daa9f56221d76d564b/zh-CN_test_0/common_voice_zh-CN_22006851.mp3',
  '/DATA10T/Cache/downloads/extracted/1e09a698341d47a1339fd58caa794de0162bda91e28f62daa9f56221d76d564b/zh-CN_test_0/common_voice_zh-CN_22115132.mp3'],
 'audio': [{'path': '/DATA10T/Cache/downloads/extracted/1e09a698341d47a1339fd58caa794de0162bda91e28f62daa9f56221d76d564b/zh-CN_test_0/common_voice_zh-CN_22069600.mp3',
   'array': array([ 4.54747351e-12,  6.13908924e-12,  2.27373675e-12, ...,
           6.41716724e-09,  2.48530796e-09, -5.68095970e-09]),
   'sampling_rate': 16000},
  {'path': '/DATA10T/Cache/downloads/extracted/1e09a698341d47a1339fd58caa794de0162bda91e28f62daa9f56221d76d564b/zh-CN_test_0/common_voice_zh-CN_22006851.mp3',
   'array': array([ 1.45519152e-10,  0.00000000e

In [66]:
test['path']

['/DATA10T/Cache/downloads/extracted/1e09a698341d47a1339fd58caa794de0162bda91e28f62daa9f56221d76d564b/zh-CN_test_0/common_voice_zh-CN_22069600.mp3',
 '/DATA10T/Cache/downloads/extracted/1e09a698341d47a1339fd58caa794de0162bda91e28f62daa9f56221d76d564b/zh-CN_test_0/common_voice_zh-CN_22006851.mp3',
 '/DATA10T/Cache/downloads/extracted/1e09a698341d47a1339fd58caa794de0162bda91e28f62daa9f56221d76d564b/zh-CN_test_0/common_voice_zh-CN_22115132.mp3']

In [67]:
transcription = asrinference_path(test['path'], model, samplerate=16_000, processor=processor, device='cuda')
transcription

Input: <class 'transformers.feature_extraction_utils.BatchFeature'>
Output logits shape: torch.Size([3, 275, 4495])


['宋朝末年年间定区粉令为', '建见行动影不变', '二十一年去世']

In [69]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    test_data=dataset[picks]
    df = pd.DataFrame(test_data)
    display(HTML(df.to_html()))
    return test_data

In [70]:
test_data = show_random_elements(dataset_zh, num_examples=5)

,path,audio,sentence
0,/DATA10T/Cache/downloads/extracted/1e09a698341d47a1339fd58caa794de0162bda91e28f62daa9f56221d76d564b/zh-CN_test_0/common_voice_zh-CN_22153652.mp3,"{'path': '/DATA10T/Cache/downloads/extracted/1e09a698341d47a1339fd58caa794de0162bda91e28f62daa9f56221d76d564b/zh-CN_test_0/common_voice_zh-CN_22153652.mp3', 'array': [0.0, -2.764863893389702e-10, 1.0186340659856796e-10, -6.548361852765083e-11, -1.6007106751203537e-10, -1.3096723705530167e-10, -5.093170329928398e-11, -5.093170329928398e-11, 3.637978807091713e-11, -1.0186340659856796e-10, 5.820766091346741e-11, 5.093170329928398e-11, -6.912159733474255e-11, 1.1641532182693481e-10, -7.275957614183426e-11, -2.9103830456733704e-10, -2.764863893389702e-10, -1.8917489796876907e-10, -7.275957614183426e-11, 0.0, -1.4551915228366852e-11, -7.275957614183426e-11, 4.3655745685100555e-11, 0.0, -7.275957614183426e-11, -1.3096723705530167e-10, -2.3283064365386963e-10, 1.0186340659856796e-10, 5.820766091346741e-11, -3.637978807091713e-11, -7.275957614183426e-11, 7.275957614183426e-11, 2.9103830456733704e-11, -4.729372449219227e-11, -5.820766091346741e-11, -1.7462298274040222e-10, -3.7834979593753815e-10, -1.6916601452976465e-10, -1.6007106751203537e-10, -7.275957614183426e-12, -1.7462298274040222e-10, -1.0186340659856796e-10, -2.9103830456733704e-11, -5.820766091346741e-11, -1.4551915228366852e-11, -1.6007106751203537e-10, 0.0, -8.731149137020111e-11, 2.9103830456733704e-10, 1.3096723705530167e-10, 1.8917489796876907e-10, 2.0372681319713593e-10, 2.9103830456733704e-11, 0.0, -1.8917489796876907e-10, 3.637978807091713e-11, -5.820766091346741e-11, -5.820766091346741e-11, 1.1641532182693481e-10, 3.4924596548080444e-10, -3.637978807091713e-11, -5.820766091346741e-11, -2.9103830456733704e-10, 5.820766091346741e-11, 5.820766091346741e-11, -4.3655745685100555e-11, 0.0, 5.820766091346741e-11, -1.1641532182693481e-10, 1.7462298274040222e-10, 2.9103830456733704e-11, -4.3655745685100555e-10, -2.9103830456733704e-10, -2.1100277081131935e-10, -3.8562575355172157e-10, -1.2369127944111824e-10, -1.382431946694851e-10, -6.548361852765083e-10, 2.582964953035116e-10, 8.949427865445614e-10, 7.312337402254343e-10, 1.0113581083714962e-09, 9.64064383879304e-10, -4.3655745685100555e-10, 3.2014213502407074e-10, 2.3283064365386963e-10, -4.911271389573812e-10, 3.3287506084889174e-10, 3.7471181713044643e-10, -1.4915713109076023e-10, 5.311449058353901e-10, 1.2350938050076365e-09, -1.4915713109076023e-10, -6.766640581190586e-10, -1.178705133497715e-09, -2.8362592274788767e-09, -1.3642420526593924e-09, -3.637978807091713e-11, 8.385541150346398e-10, 1.382431946694851e-09, ...], 'sampling_rate': 16000}",高级工程师。
1,/DATA10T/Cache/downloads/extracted/1e09a698341d47a1339fd58caa794de0162bda91e28f62daa9f56221d76d564b/zh-CN_test_0/common_voice_zh-CN_22050053.mp3,"{'path': '/DATA10T/Cache/downloads/extracted/1e09a698341d47a1339fd58caa794de0162bda91e28f62daa9f56221d76d564b/zh-CN_test_0/common_voice_zh-CN_22050053.mp3', 'array': [7.639755494892597e-11, 2.1827872842550278e-11, -5.093170329928398e-11, -5.093170329928398e-11, -2.1827872842550278e-11, -5.820766091346741e-11, -2.1827872842550278e-11, -4.3655745685100555e-11, 7.275957614183426e-11, -2.9103830456733704e-11, 7.275957614183426e-11, 2.1827872842550278e-11, 4.3655745685100555e-11, 1.2005330063402653e-10, 1.0186340659856796e-10, 7.821654435247183e-11, 1.8189894035458565e-10, 7.275957614183426e-12, -5.820766091346741e-11, -7.275957614183426e-11, 2.9103830456733704e-11, -3.2741809263825417e-11, -1.0913936421275139e-11, 5.4569682106375694e-11, 5.820766091346741e-11, 5.820766091346741e-11, 1.4551915228366852e-10, 5.820766091346741e-11, -5.820766091346741e-11, -5.820766091346741e-11, -5.093170329928398e-11, -1.5279510989785194e-10, -5.275069270282984e-11, -1.5643308870494366e-10, -2.1100277081131935e-10, -7.275957614183426e-11, -1.127773430198431e-10, 1.4551915228366852e-11, -1.1823431123048067e-11, 9.276845958083868e-11, 1.6370904631912708e-10, 8.003553375601768e-11, -1.

In [71]:
transcription = asrinference_path(test_data['path'], model, samplerate=16_000, processor=processor, device='cuda')
transcription

Input: <class 'transformers.feature_extraction_utils.BatchFeature'>
Output logits shape: torch.Size([5, 490, 4495])


['高籍共程师',
 '他们在此地前射大学的别书',
 '德知消需后的目府攻开自政感蟹状于赏监表阳星撰筑共脑',
 '也称格洛斯特或格洛斯特花蛛',
 '乔致市现在所在的孩角曾被当地马来人称为单容']

In [73]:
processor.tokenizer.set_target_lang("eng")

In [74]:
test_data = show_random_elements(dataset_en, num_examples=5)

,path,audio,sentence
0,/DATA10T/Cache/downloads/extracted/ba5c265b423696aeece652c80df565d72d9f7c1cc9b807e127f44f989150f409/en_test_0/common_voice_en_19546213.mp3,"{'path': '/DATA10T/Cache/downloads/extracted/ba5c265b423696aeece652c80df565d72d9f7c1cc9b807e127f44f989150f409/en_test_0/common_voice_en_19546213.mp3', 'array': [5.6843418860808015e-12, 1.8189894035458565e-12, 9.549694368615746e-12, -5.002220859751105e-12, 3.637978807091713e-12, -4.547473508864641e-13, 7.73070496506989e-12, 6.821210263296962e-12, 4.547473508864641e-13, 1.0345502232667059e-11, -2.7284841053187847e-12, 6.366462912410498e-12, 2.1600499167107046e-12, 5.6843418860808015e-12, -9.43600753089413e-12, 4.547473508864641e-13, -9.094947017729282e-13, 1.1823431123048067e-11, -4.547473508864641e-12, 1.0913936421275139e-11, 2.2737367544323206e-12, 2.3305801732931286e-11, -6.366462912410498e-12, -8.86757334228605e-12, 3.319655661471188e-11, -5.4569682106375694e-12, -8.640199666842818e-12, -2.2282620193436742e-11, -2.4101609596982598e-11, -2.2737367544323206e-12, 2.955857780762017e-11, 6.821210263296962e-13, 7.617018127348274e-11, 4.206412995699793e-11, -7.275957614183426e-12, -5.002220859751105e-11, 1.9099388737231493e-11, -2.1600499167107046e-11, 6.730260793119669e-11, -5.638867150992155e-11, -8.230927051045e-11, -1.8189894035458565e-11, 1.800799509510398e-10, 1.4279066817834973e-10, -6.912159733474255e-11, 1.837179297581315e-10, -9.367795428261161e-11, -2.6193447411060333e-10, -1.8053469830192626e-10, -2.1600499167107046e-10, 8.049028110690415e-11, 4.4929038267582655e-10, -1.3096723705530167e-10, 3.9835867937654257e-10, 5.688889359589666e-10, 1.9554136088117957e-11, -3.5129232855979353e-10, -1.2232703738845885e-10, 1.6370904631912708e-11, -2.000888343900442e-10, -2.721662895055488e-10, -5.043148121330887e-10, -1.1095835361629725e-10, 1.0795702110044658e-09, 1.1466454452602193e-09, 6.605205271625891e-10, 1.0809344530571252e-09, -2.6057023205794394e-10, -1.6693775251042098e-09, -1.4306351658888161e-09, -4.001776687800884e-10, -1.4097167877480388e-09, 2.603201210149564e-09, -3.005879989359528e-10, -6.621121428906918e-10, 3.4976892493432388e-09, 5.147740012034774e-10, -8.799361239653081e-10, -3.894911060342565e-09, 5.459241947392002e-10, -1.3751559890806675e-09, -1.529542714706622e-09, -1.2310010788496584e-09, -2.153115019609686e-09, 3.4463027986930683e-09, 9.209088602801785e-09, 1.0645635484252125e-09, 1.3949943422630895e-09, 1.3926864994573407e-08, -1.16006049211137e-09, -2.8458089218474925e-09, -1.1051554338337155e-08, -1.9053914002142847e-08, -1.2194504961371422e-08, 1.1584688763832673e-08, 1.9682147467392497e-08, 6.7292376115801744e-09, 2.2318090486805886e-08, 3.7161953514441848e-09, 2.1746018319390714e-09, ...], 'sampling_rate': 16000}",the lyceum presents one of sheffield's annual christmas pantomimes
1,/DATA10T/Cache/downloads/extracted/ba5c265b423696aeece652c80df565d72d9f7c1cc9b807e127f44f989150f409/en_test_0/common_voice_en_18948246.mp3,"{'path': '/DATA10T/Cache/downloads/extracted/ba5c265b423696aeece652c80df565d72d9f7c1cc9b807e127f44f989150f409/en_test_0/common_voice_en_18948246.mp3', 'array': [-2.329340604949326e-21, 1.2176098616780567e-20, -1.4293680984916318e-21, 1.5881867761018131e-21, 8.258571235729428e-21, -5.161607022330893e-21, -2.6469779601696886e-21, 2.541098841762901e-21, 3.441071348220595e-21, -1.4823076576950256e-21, 3.282252670610414e-21, -1.164670302474663e-21, -1.0323214044661785e-21, -8.999725064576941e-22, 9.952637130238029e-21, 5.399835038746165e-21, -2.064642808932357e-21, -5.293955920339377e-22, 1.5881867761018131e-21, -7.146840492458159e-22, 1.1408475008331358e-20, -1.641126335305207e-21, -1.7999450129153882e-21, -3.5998900258307764e-21, -5.6910026143648304e-21, -9.317362419797304e-21, -3.22931311140702e-21, 3.5998900258307764e-21, -1.6940658945086007e-21, 5.770411953169921e-21, -1.6940658945086007e-21, -3.3881317890172014e-21, -3.3881317890172014e-21, -2.541098841762901e-21, -2.117582368135751e-22, 4.446922973085077e-21, 7.093900933254

In [75]:
transcription = asrinference_path(test_data['path'], model, samplerate=16_000, processor=processor, device='cuda')
transcription

Input: <class 'transformers.feature_extraction_utils.BatchFeature'>
Output logits shape: torch.Size([5, 380, 4495])


['н0[$2ò$şq<unk>9080qò82q02<unk>&ê8ê0<unk>$0[н8hнş[ò9$8时h8<unk>hq&2时$时8',
 '8&şh9&<unk>h8$q<unk>2[<unk>0н$q8<unk>29&<unk>92时qh9[<unk>hę0',
 '<unk>ê0qê0$ę2&&ê0òh[[h[<unk>9hq2ş&2<unk>&ê08&29н2<unk>q&ê08&900&hqн$q&2&ê0òh9',
 '8&2q0$时<unk>[0时0q&8<unk>92时&ê0q02[$&ê$òhqн<unk>92q8hę0<unk>090<unk>2şqн',
 '&ê0[2òh[8q0h9&ê0时$qнêh<unk>090<unk>29&0ннh时hę08&2&ê0$9ê2ş808']

In [76]:
model.load_adapter("eng")

adapter.eng.safetensors:   0%|          | 0.00/9.43M [00:00<?, ?B/s]

In [77]:
transcription = asrinference_path(test_data['path'], model, samplerate=16_000, processor=processor, device='cuda')
transcription

Input: <class 'transformers.feature_extraction_utils.BatchFeature'>
Output logits shape: torch.Size([5, 380, 154])


["te liosium presents one of the sheffield's annoal christmas pantomims",
 'stuwart was involved in sport from an early age',
 'when he got the call alli ran out of the store down the street and into the car',
 'stone implements from the neolythic and bronzeage were found',
 'the locals near the mine have reported damages to their houses']

In [52]:
oneexample

{'path': '/DATA10T/Cache/downloads/extracted/1e09a698341d47a1339fd58caa794de0162bda91e28f62daa9f56221d76d564b/zh-CN_test_0/common_voice_zh-CN_22157117.mp3',
 'audio': {'path': '/DATA10T/Cache/downloads/extracted/1e09a698341d47a1339fd58caa794de0162bda91e28f62daa9f56221d76d564b/zh-CN_test_0/common_voice_zh-CN_22157117.mp3',
  'array': array([ 1.74622983e-10,  3.05590220e-10, -1.60071068e-10, ...,
          1.95016852e-04,  6.61519589e-05, -1.74502173e-04]),
  'sampling_rate': 16000},
 'sentence': '该文物保护单位由汪清县文物管理所管理。 '}

In [11]:
inputs = processor(en_sample, sampling_rate=16_000, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs).logits

ids = torch.argmax(outputs, dim=-1)[0]
transcription = processor.decode(ids)

In [12]:
transcription

'joe keton disapproved of films and buster also had reservations about the media'

In [13]:
processor.tokenizer.set_target_lang("fra")
model.load_adapter("fra")


adapter.fra.safetensors:   0%|          | 0.00/10.2M [00:00<?, ?B/s]

In [14]:
inputs = processor(fr_sample, sampling_rate=16_000, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs).logits

ids = torch.argmax(outputs, dim=-1)[0]
transcription = processor.decode(ids)
transcription

"ce dernier est volé tout au long de l'histoire romaine"

In [15]:
processor.tokenizer.vocab.keys()

dict_keys(['abi', 'abk', 'abp', 'aca', 'acd', 'ace', 'acf', 'ach', 'acn', 'acr', 'acu', 'ade', 'adh', 'adj', 'adx', 'aeu', 'afr', 'agd', 'agg', 'agn', 'agr', 'agu', 'agx', 'aha', 'ahk', 'aia', 'aka', 'akb', 'ake', 'akp', 'alj', 'alp', 'alt', 'alz', 'ame', 'amf', 'amh', 'ami', 'amk', 'ann', 'any', 'aoz', 'apb', 'apr', 'ara', 'arl', 'asa', 'asg', 'asm', 'ast', 'ata', 'atb', 'atg', 'ati', 'atq', 'ava', 'avn', 'avu', 'awa', 'awb', 'ayo', 'ayr', 'ayz', 'azb', 'azg', 'azj-script_cyrillic', 'azj-script_latin', 'azz', 'bak', 'bam', 'ban', 'bao', 'bas', 'bav', 'bba', 'bbb', 'bbc', 'bbo', 'bcc-script_arabic', 'bcc-script_latin', 'bcl', 'bcw', 'bdg', 'bdh', 'bdq', 'bdu', 'bdv', 'beh', 'bel', 'bem', 'ben', 'bep', 'bex', 'bfa', 'bfo', 'bfy', 'bfz', 'bgc', 'bgq', 'bgr', 'bgt', 'bgw', 'bha', 'bht', 'bhz', 'bib', 'bim', 'bis', 'biv', 'bjr', 'bjv', 'bjw', 'bjz', 'bkd', 'bkv', 'blh', 'blt', 'blx', 'blz', 'bmq', 'bmr', 'bmu', 'bmv', 'bng', 'bno', 'bnp', 'boa', 'bod', 'boj', 'bom', 'bor', 'bos', 'bov', 'b

In [ ]:
#cmn https://dl.fbaipublicfiles.com/mms/misc/language_coverage_mms.html

In [17]:
processor.tokenizer.set_target_lang("cmn-script_simplified")
model.load_adapter("cmn-script_simplified")

(…)dapter.cmn-script_simplified.safetensors:   0%|          | 0.00/31.7M [00:00<?, ?B/s]

In [18]:
inputs = processor(zh_sample, sampling_rate=16_000, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs).logits

ids = torch.argmax(outputs, dim=-1)[0]
transcription = processor.decode(ids)
transcription

'法'

In [22]:
processor.tokenizer.vocab.keys()

dict_keys(['abi', 'abk', 'abp', 'aca', 'acd', 'ace', 'acf', 'ach', 'acn', 'acr', 'acu', 'ade', 'adh', 'adj', 'adx', 'aeu', 'afr', 'agd', 'agg', 'agn', 'agr', 'agu', 'agx', 'aha', 'ahk', 'aia', 'aka', 'akb', 'ake', 'akp', 'alj', 'alp', 'alt', 'alz', 'ame', 'amf', 'amh', 'ami', 'amk', 'ann', 'any', 'aoz', 'apb', 'apr', 'ara', 'arl', 'asa', 'asg', 'asm', 'ast', 'ata', 'atb', 'atg', 'ati', 'atq', 'ava', 'avn', 'avu', 'awa', 'awb', 'ayo', 'ayr', 'ayz', 'azb', 'azg', 'azj-script_cyrillic', 'azj-script_latin', 'azz', 'bak', 'bam', 'ban', 'bao', 'bas', 'bav', 'bba', 'bbb', 'bbc', 'bbo', 'bcc-script_arabic', 'bcc-script_latin', 'bcl', 'bcw', 'bdg', 'bdh', 'bdq', 'bdu', 'bdv', 'beh', 'bel', 'bem', 'ben', 'bep', 'bex', 'bfa', 'bfo', 'bfy', 'bfz', 'bgc', 'bgq', 'bgr', 'bgt', 'bgw', 'bha', 'bht', 'bhz', 'bib', 'bim', 'bis', 'biv', 'bjr', 'bjv', 'bjw', 'bjz', 'bkd', 'bkv', 'blh', 'blt', 'blx', 'blz', 'bmq', 'bmr', 'bmu', 'bmv', 'bng', 'bno', 'bnp', 'boa', 'bod', 'boj', 'bom', 'bor', 'bos', 'bov', 'b

In [19]:
zh_sample = next(iter(stream_data))["audio"]["array"]

Reading metadata...: 10581it [00:00, 16851.05it/s]


In [20]:
def inference(model, data_sample, sampling_rate=16_000, device='cuda'):
    inputs = processor(data_sample, sampling_rate=sampling_rate, return_tensors="pt")
    model=model.to(device)
    inputs=inputs.to(device)

    with torch.no_grad():
        outputs = model(**inputs).logits

    ids = torch.argmax(outputs, dim=-1)[0]
    transcription = processor.decode(ids)
    return transcription

In [21]:
transcription=inference(model, zh_sample)
transcription

'法'